In [277]:
! pip install awswrangler scikit-learn plotly


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [278]:
import pandas as pd
import numpy as np
# from pycaret.time_series import *
import matplotlib.pyplot as plt
import awswrangler as wr
import boto3
# from pycaret import *
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, cross_validate
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, Input


In [279]:
boto3.setup_default_session(profile_name='aisha_younas_fellow_dssgx_24')
bucket = "dssgx-munich-2024-bavarian-forest"
raw_data_folder = "raw-data"
preprocessed_data_folder = "preprocessed_data"

In [280]:
def load_csv_files_from_aws_s3(path: str, **kwargs) -> pd.DataFrame:
    """Loads individual or multiple CSV files from an AWS S3 bucket.
    Args:
        path (str): The path to the CSV files on AWS S3.
        **kwargs: Additional arguments to pass to the read_csv function.
    Returns:
        pd.DataFrame: The DataFrame containing the data from the CSV files.
    """
    df = wr.s3.read_csv(path=path, **kwargs)
    return df
df = load_csv_files_from_aws_s3(
    path="s3://dssgx-munich-2024-bavarian-forest/preprocessed_data/joined_sensor_weather_visitorcenter_2016-2024.csv"
)
df.head()

c:\Users\workshop\.pyenv-win-venv\envs\bf_dssgx\lib\site-packages\awswrangler\s3\_read_text_core.py:115: DtypeWarning: Columns (98,99,100,101,109,110,111,112,113,114,115,116,117,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df: pd.DataFrame = parser_func(f, **pandas_kwargs)


,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,...,Racheldiensthuette_geoeffnet,Waldschmidthaus_geoeffnet,Falkensteinschutzhaus_geoeffnet,Schwellhaeusl_geoeffnet,Temperatur,Niederschlagsmenge,Schneehoehe,GS mit,GS max,Total
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [281]:
print(list(df.columns))


['Time', 'Bayerisch Eisenstein IN', 'Bayerisch Eisenstein OUT', 'Brechhäuslau IN', 'Brechhäuslau OUT', 'Deffernik IN', 'Deffernik OUT', 'Diensthüttenstraße IN', 'Diensthüttenstraße OUT', 'Felswandergebiet IN', 'Felswandergebiet OUT', 'Ferdinandsthal IN', 'Ferdinandsthal OUT', 'Fredenbrücke IN', 'Fredenbrücke OUT', 'Gfäll IN', 'Gfäll OUT', 'Gsenget IN', 'Gsenget OUT', 'Klingenbrunner Wald IN', 'Klingenbrunner Wald OUT', 'Klosterfilz IN', 'Klosterfilz OUT', 'Racheldiensthütte IN', 'Racheldiensthütte OUT', 'Sagwassersäge IN', 'Sagwassersäge OUT', 'Scheuereck IN', 'Scheuereck OUT', 'Schillerstraße IN', 'Schillerstraße OUT', 'Schwarzbachbrücke IN', 'Schwarzbachbrücke OUT', 'Falkenstein 2 OUT', 'Falkenstein 2 IN', 'Lusen 2 IN', 'Lusen 2 OUT', 'Lusen 3 IN', 'Lusen 3 OUT', 'Waldhausreibe IN', 'Waldhausreibe OUT', 'Waldspielgelände IN', 'Waldspielgelände OUT', 'Wistlberg IN', 'Wistlberg OUT', 'Bucina MERGED IN', 'Bucina MERGED OUT', 'Falkenstein 1 MERGED IN', 'Falkenstein 1 MERGED OUT', 'Lusen 

In [282]:
# Specify the columns to use
columns_to_use = [
'Time',  'Bayerisch Eisenstein IN',  'Bayerisch Eisenstein OUT',  'Brechhäuslau IN',  'Brechhäuslau OUT',  
'Deffernik IN',  'Deffernik OUT',  'Diensthüttenstraße IN',  'Diensthüttenstraße OUT',  'Felswandergebiet IN',  
'Felswandergebiet OUT',  'Ferdinandsthal IN',  'Ferdinandsthal OUT',  'Fredenbrücke IN',  'Fredenbrücke OUT',  
'Gfäll IN',  'Gfäll OUT',  'Gsenget IN',  'Gsenget OUT',  'Klingenbrunner Wald IN',  'Klingenbrunner Wald OUT',  
'Klosterfilz IN',  'Klosterfilz OUT',  'Racheldiensthütte IN',  'Racheldiensthütte OUT',  'Sagwassersäge IN',  
'Sagwassersäge OUT',  'Scheuereck IN',  'Scheuereck OUT',  'Schillerstraße IN',  'Schillerstraße OUT',  
'Schwarzbachbrücke IN',  'Schwarzbachbrücke OUT',  'Falkenstein 2 OUT',  'Falkenstein 2 IN',  'Lusen 2 IN',  
'Lusen 2 OUT',  'Lusen 3 IN',  'Lusen 3 OUT',  'Waldhausreibe IN',  'Waldhausreibe OUT',  'Waldspielgelände IN',  
'Waldspielgelände OUT',  'Wistlberg IN',  'Wistlberg OUT',  'Bucina MERGED IN',  'Bucina MERGED OUT',  
'Falkenstein 1 MERGED IN',  'Falkenstein 1 MERGED OUT',  'Lusen 1 MERGED IN',  'Lusen 1 MERGED OUT',  
'Trinkwassertalsperre MERGED IN',  'Trinkwassertalsperre MERGED OUT',  'working_sensors',  
'traffic_abs',  'sum_IN_abs',  'sum_OUT_abs',  'Temperature (°C)',  'Relative Humidity (%)',  
'Precipitation (mm)',  'Wind Speed (km/h)',  'Sunshine Duration (min)',  'Tag',  'Monat',  'Jahr',  
'Wochentag',  'Wochenende',  'Jahreszeit',  'Laubfärbung',  'Schulferien_Bayern',  'Schulferien_CZ',  
'Feiertag_Bayern',  'Feiertag_CZ',  'HEH_geoeffnet',  'HZW_geoeffnet',  'WGM_geoeffnet',  
'Lusenschutzhaus_geoeffnet',  'Racheldiensthuette_geoeffnet',  'Falkensteinschutzhaus_geoeffnet',  
'Schwellhaeusl_geoeffnet'
]


In [283]:
# Filter the dataframe to only include the specified columns
df = df[columns_to_use]

# Display the first few rows to ensure the data is loaded correctly
df.head()

,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,...,Schulferien_CZ,Feiertag_Bayern,Feiertag_CZ,HEH_geoeffnet,HZW_geoeffnet,WGM_geoeffnet,Lusenschutzhaus_geoeffnet,Racheldiensthuette_geoeffnet,Falkensteinschutzhaus_geoeffnet,Schwellhaeusl_geoeffnet
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [284]:
# CREATE NEW REGION VARIABLE

# Remove MERGED from column names with this unnecessary label
df.columns = df.columns.str.replace(' MERGED', '', regex=False)

# Create a dictionary for mapping
location_mapping = {
    'Bayerisch Eisenstein IN': 'Falkenstein-Schwellhäusl',
    'Bayerisch Eisenstein OUT': 'Falkenstein-Schwellhäusl', 
    'Brechhäuslau IN': 'Falkenstein-Schwellhäusl', 
    'Brechhäuslau OUT': 'Falkenstein-Schwellhäusl', 
    'Deffernik IN': 'Falkenstein-Schwellhäusl',
    'Deffernik OUT': 'Falkenstein-Schwellhäusl',
    'Falkenstein 1 IN': 'Nationalparkzentrum Falkenstein', 
    'Falkenstein 1 OUT': 'Nationalparkzentrum Falkenstein',
    'Falkenstein 2 IN': 'Nationalparkzentrum Falkenstein', 
    'Falkenstein 2 OUT': 'Nationalparkzentrum Falkenstein',
    'Ferdinandsthal IN': 'Falkenstein-Schwellhäusl', 
    'Ferdinandsthal OUT': 'Falkenstein-Schwellhäusl', 
    'Gsenget IN': 'Scheuereck-Schachten-Trinkwassertalsperre', 
    'Gsenget OUT': 'Scheuereck-Schachten-Trinkwassertalsperre', 
    'Scheuereck IN': 'Scheuereck-Schachten-Trinkwassertalsperre',
    'Scheuereck OUT': 'Scheuereck-Schachten-Trinkwassertalsperre', 
    'Schillerstraße IN': 'Falkenstein-Schwellhäusl', 
    'Schillerstraße OUT': 'Falkenstein-Schwellhäusl', 
    'Trinkwassertalsperre IN': 'Scheuereck-Schachten-Trinkwassertalsperre',
    'Trinkwassertalsperre OUT': 'Scheuereck-Schachten-Trinkwassertalsperre',
    'Bucina IN': 'Lusen-Mauth-Finsterau',
    'Bucina OUT': 'Lusen-Mauth-Finsterau', 
    'Diensthüttenstraße IN': 'Rachel-Spiegelau', 
    'Diensthüttenstraße OUT': 'Rachel-Spiegelau',
    'Felswandergebiet IN': 'Lusen-Mauth-Finsterau', 
    'Felswandergebiet OUT': 'Lusen-Mauth-Finsterau',
    'Fredenbrücke IN': 'Lusen-Mauth-Finsterau', 
    'Fredenbrücke OUT': 'Lusen-Mauth-Finsterau', 
    'Gfäll IN': 'Rachel-Spiegelau', 
    'Gfäll OUT': 'Rachel-Spiegelau', 
    'Klingenbrunner Wald IN': 'Rachel-Spiegelau', 
    'Klingenbrunner Wald OUT': 'Rachel-Spiegelau', 
    'Klosterfilz IN': 'Rachel-Spiegelau', 
    'Klosterfilz OUT': 'Rachel-Spiegelau',
    'Lusen 1 IN': 'Nationalparkzentrum Lusen', 
    'Lusen 1 OUT': 'Nationalparkzentrum Lusen', 
    'Lusen 2 IN': 'Nationalparkzentrum Lusen',
    'Lusen 2 OUT': 'Nationalparkzentrum Lusen', 
    'Lusen 3 IN': 'Nationalparkzentrum Lusen', 
    'Lusen 3 OUT': 'Nationalparkzentrum Lusen',
    'Racheldiensthütte IN': 'Rachel-Spiegelau', 
    'Racheldiensthütte OUT': 'Rachel-Spiegelau',
    'Schwarzbachbrücke IN': 'Lusen-Mauth-Finsterau', 
    'Schwarzbachbrücke OUT': 'Lusen-Mauth-Finsterau', 
    'Waldhausreibe IN': 'Lusen-Mauth-Finsterau', 
    'Waldhausreibe OUT': 'Lusen-Mauth-Finsterau', 
    'Waldspielgelände IN': 'Rachel-Spiegelau', 
    'Waldspielgelände OUT': 'Rachel-Spiegelau', 
    'Wistlberg IN': 'Lusen-Mauth-Finsterau', 
    'Wistlberg OUT': 'Lusen-Mauth-Finsterau', 
    'Sagwassersäge IN': 'Lusen-Mauth-Finsterau',
    'Sagwassersäge OUT': 'Lusen-Mauth-Finsterau'
}

# Extract unique regions
regions = set(location_mapping.values())

# Iterate over each region
for region in regions:
    # Filter the keys in location_mapping that belong to the current region
    region_in_columns = [col for col in location_mapping if location_mapping[col] == region and ' IN' in col]
    region_out_columns = [col for col in location_mapping if location_mapping[col] == region and ' OUT' in col]

    # Sum the values for all IN columns of the current region, while retaining NaN where all are NaN
    df[f'{region} IN'] = df[region_in_columns].sum(axis=1, min_count=1)
    
    # Sum the values for all OUT columns of the current region, while retaining NaN where all are NaN
    df[f'{region} OUT'] = df[region_out_columns].sum(axis=1, min_count=1)

# Display the updated DataFrame
df.tail()

# EXPLANATION OF LOOPING FUNCTION BELOW
#min_count=1 in sum():
#The sum(axis=1, min_count=1) method ensures that if all values being summed are NaN, the result will be NaN.
#If at least one value is not NaN, it will compute the sum, ignoring the NaN values.
#Explanation:
#min_count=1: This parameter in the sum() function specifies the minimum number of non-NaN values required to perform the summation. If the count of non-NaN values is less than min_count, the result will be NaN.
#Result: The DataFrame will have the new region columns that sum the sensors while retaining NaN if all sensors in a region are NaN for a given row.

,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,...,Nationalparkzentrum Lusen IN,Nationalparkzentrum Lusen OUT,Nationalparkzentrum Falkenstein IN,Nationalparkzentrum Falkenstein OUT,Lusen-Mauth-Finsterau IN,Lusen-Mauth-Finsterau OUT,Falkenstein-Schwellhäusl IN,Falkenstein-Schwellhäusl OUT,Scheuereck-Schachten-Trinkwassertalsperre IN,Scheuereck-Schachten-Trinkwassertalsperre OUT
78907,2024-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78908,2024-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78909,2024-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78910,2024-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78911,2024-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [285]:
print(df.columns.tolist())

['Time', 'Bayerisch Eisenstein IN', 'Bayerisch Eisenstein OUT', 'Brechhäuslau IN', 'Brechhäuslau OUT', 'Deffernik IN', 'Deffernik OUT', 'Diensthüttenstraße IN', 'Diensthüttenstraße OUT', 'Felswandergebiet IN', 'Felswandergebiet OUT', 'Ferdinandsthal IN', 'Ferdinandsthal OUT', 'Fredenbrücke IN', 'Fredenbrücke OUT', 'Gfäll IN', 'Gfäll OUT', 'Gsenget IN', 'Gsenget OUT', 'Klingenbrunner Wald IN', 'Klingenbrunner Wald OUT', 'Klosterfilz IN', 'Klosterfilz OUT', 'Racheldiensthütte IN', 'Racheldiensthütte OUT', 'Sagwassersäge IN', 'Sagwassersäge OUT', 'Scheuereck IN', 'Scheuereck OUT', 'Schillerstraße IN', 'Schillerstraße OUT', 'Schwarzbachbrücke IN', 'Schwarzbachbrücke OUT', 'Falkenstein 2 OUT', 'Falkenstein 2 IN', 'Lusen 2 IN', 'Lusen 2 OUT', 'Lusen 3 IN', 'Lusen 3 OUT', 'Waldhausreibe IN', 'Waldhausreibe OUT', 'Waldspielgelände IN', 'Waldspielgelände OUT', 'Wistlberg IN', 'Wistlberg OUT', 'Bucina IN', 'Bucina OUT', 'Falkenstein 1 IN', 'Falkenstein 1 OUT', 'Lusen 1 IN', 'Lusen 1 OUT', 'Trink

In [286]:
def load_csv_files_from_aws_s3(path: str, **kwargs) -> pd.DataFrame:
    """Loads individual or multiple CSV files from an AWS S3 bucket.
    Args:
        path (str): The path to the CSV files on AWS S3.
        **kwargs: Additional arguments to pass to the read_csv function.
    Returns:
        pd.DataFrame: The DataFrame containing the data from the CSV files.
    """
    df = wr.s3.read_csv(path=path, **kwargs)
    return df
df_newfeatures = load_csv_files_from_aws_s3(
    path="s3://dssgx-munich-2024-bavarian-forest/preprocessed_data/holidays_deltaweather_features_df.csv"
)
df_newfeatures.head()

,Unnamed: 0,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,...,GS max,Total,Date,Distance_to_Nearest_Holiday_Bayern,Distance_to_Nearest_Holiday_CZ,ZScore_Daily_Max_Temperature (°C),ZScore_Daily_Max_Relative Humidity (%),ZScore_Daily_Max_Precipitation (mm),ZScore_Daily_Max_Wind Speed (km/h),ZScore_Daily_Max_Sunshine Duration (min)
0,0,2017-01-01 00:00:00,NaN,NaN,257.0,412.0,NaN,NaN,NaN,NaN,...,345.0,1.0,2017-01-01,0,0,NaN,NaN,NaN,NaN,NaN
1,1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.0,1.0,2017-01-01,0,0,NaN,NaN,NaN,NaN,NaN
2,2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.0,1.0,2017-01-01,0,0,NaN,NaN,NaN,NaN,NaN
3,3,2017-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.0,1.0,2017-01-01,0,0,NaN,NaN,NaN,NaN,NaN
4,4,2017-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,345.0,1.0,2017-01-01,0,0,NaN,NaN,NaN,NaN,NaN


In [287]:
df['Time'] = pd.to_datetime(df['Time'])
df_newfeatures['Time'] = pd.to_datetime(df_newfeatures['Time'])

# Step 2: Select the columns you want to add from df_newfeatures
columns_to_add = [
    'ZScore_Daily_Max_Temperature (°C)',
    'ZScore_Daily_Max_Relative Humidity (%)',
    'ZScore_Daily_Max_Precipitation (mm)',
    'ZScore_Daily_Max_Wind Speed (km/h)',
    'ZScore_Daily_Max_Sunshine Duration (min)',
    'Distance_to_Nearest_Holiday_Bayern',
    'Distance_to_Nearest_Holiday_CZ'
]

# Ensure that the selected columns exist in df_newfeatures
selected_columns = [col for col in columns_to_add if col in df_newfeatures.columns]

# Step 3: Merge df with df_newfeatures on 'Time' and add the selected columns
df = pd.merge(df, df_newfeatures[['Time'] + selected_columns], on='Time', how='left')

# Optionally, you can display the merged dataframe
df.head()

,Time,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,...,Falkenstein-Schwellhäusl OUT,Scheuereck-Schachten-Trinkwassertalsperre IN,Scheuereck-Schachten-Trinkwassertalsperre OUT,ZScore_Daily_Max_Temperature (°C),ZScore_Daily_Max_Relative Humidity (%),ZScore_Daily_Max_Precipitation (mm),ZScore_Daily_Max_Wind Speed (km/h),ZScore_Daily_Max_Sunshine Duration (min),Distance_to_Nearest_Holiday_Bayern,Distance_to_Nearest_Holiday_CZ
0,2016-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
df['Time'] = pd.to_datetime(df['Time'])  # Convert to datetime if in columns
df.set_index('Time', inplace=True)
df["Hour"] = df.index.hour  # Now this will work

# Specify the data types
dtype_dict = {
    'Time': 'datetime64[ns]',
    'traffic_abs': 'float64',
    'Temperature (°C)': 'float64',
    'Relative Humidity (%)': 'float64',
    'Precipitation (mm)': 'float64',
    'Wind Speed (km/h)': 'float64',
    'Sunshine Duration (min)': 'float64',
    'Monat': 'float64',
    'Wochentag': 'category',
    'Wochenende': 'category',
    'Jahreszeit': 'category',
    'Laubfärbung': 'category',
    'Feiertag_Bayern': 'category',
    'Feiertag_CZ': 'category',
    'HEH_geoeffnet': 'category',
    'HZW_geoeffnet': 'category',
    'WGM_geoeffnet': 'category',
    'Lusenschutzhaus_geoeffnet': 'category',
    'Racheldiensthuette_geoeffnet': 'category',
    'Falkensteinschutzhaus_geoeffnet': 'category',
    'Schwellhaeusl_geoeffnet': 'category',
    'Schulferien_Bayern': 'category',
    'Schulferien_CZ': 'category',
    'Jahr': 'float64',
    'working_sensors': 'float64',
    'sum_IN_abs': 'float64',
    'sum_OUT_abs': 'float64',
    'Falkenstein-Schwellhäusl IN': 'float64',
    'Rachel-Spiegelau IN': 'float64',
    'Nationalparkzentrum Falkenstein IN': 'float64',
    'Nationalparkzentrum Lusen IN': 'float64',
    'Lusen-Mauth-Finsterau IN': 'float64',
    'Scheuereck-Schachten-Trinkwassertalsperre IN': 'float64',
    'Falkenstein-Schwellhäusl OUT': 'float64',
    'Rachel-Spiegelau OUT': 'float64',
    'Nationalparkzentrum Falkenstein OUT': 'float64',
    'Nationalparkzentrum Lusen OUT': 'float64',
    'Lusen-Mauth-Finsterau OUT': 'float64',
    'Scheuereck-Schachten-Trinkwassertalsperre OUT': 'float64',
    'Bayerisch Eisenstein IN': 'float64',
    'Bayerisch Eisenstein OUT': 'float64',
    'Brechhäuslau IN': 'float64',
    'Brechhäuslau OUT': 'float64',
    'Deffernik IN': 'float64',
    'Deffernik OUT': 'float64',
    'Diensthüttenstraße IN': 'float64',
    'Diensthüttenstraße OUT': 'float64',
    'Felswandergebiet IN': 'float64',
    'Felswandergebiet OUT': 'float64',
    'Ferdinandsthal IN': 'float64',
    'Ferdinandsthal OUT': 'float64',
    'Fredenbrücke IN': 'float64',
    'Fredenbrücke OUT': 'float64',
    'Gfäll IN': 'float64',
    'Gfäll OUT': 'float64',
    'Gsenget IN': 'float64',
    'Gsenget OUT': 'float64',
    'Klingenbrunner Wald IN': 'float64',
    'Klingenbrunner Wald OUT': 'float64',
    'Klosterfilz IN': 'float64',
    'Klosterfilz OUT': 'float64',
    'Racheldiensthütte IN': 'float64',
    'Racheldiensthütte OUT': 'float64',
    'Sagwassersäge IN': 'float64',
    'Sagwassersäge OUT': 'float64',
    'Scheuereck IN': 'float64',
    'Scheuereck OUT': 'float64',
    'Schillerstraße IN': 'float64',
    'Schillerstraße OUT': 'float64',
    'Schwarzbachbrücke IN': 'float64',
    'Schwarzbachbrücke OUT': 'float64',
    'Falkenstein 2 OUT': 'float64',
    'Falkenstein 2 IN': 'float64',
    'Lusen 2 IN': 'float64',
    'Lusen 2 OUT': 'float64',
    'Lusen 3 IN': 'float64',
    'Lusen 3 OUT': 'float64',
    'Waldhausreibe IN': 'float64',
    'Waldhausreibe OUT': 'float64',
    'Waldspielgelände IN': 'float64',
    'Waldspielgelände OUT': 'float64',
    'Wistlberg IN': 'float64',
    'Wistlberg OUT': 'float64',
    'Bucina IN': 'float64',
    'Bucina OUT': 'float64',
    'Falkenstein 1 IN': 'float64',
    'Falkenstein 1 OUT': 'float64',
    'Lusen 1 IN': 'float64',
    'Lusen 1 OUT': 'float64',
    'Trinkwassertalsperre IN': 'float64',
    'Trinkwassertalsperre OUT': 'float64',
    'ZScore_Daily_Max_Temperature (°C)': 'float64',
    'ZScore_Daily_Max_Relative Humidity (%)': 'float64',
    'ZScore_Daily_Max_Precipitation (mm)': 'float64',
    'ZScore_Daily_Max_Wind Speed (km/h)': 'float64',
    'ZScore_Daily_Max_Sunshine Duration (min)': 'float64',
    'Distance_to_Nearest_Holiday_Bayern': 'float64',
    'Distance_to_Nearest_Holiday_CZ': 'float64',
    'Hour': 'category'
}

# Show the dataframe
print(df.head())


                     Bayerisch Eisenstein IN  Bayerisch Eisenstein OUT  \
Time                                                                     
2016-01-01 00:00:00                      NaN                       NaN   
2016-01-01 01:00:00                      NaN                       NaN   
2016-01-01 02:00:00                      NaN                       NaN   
2016-01-01 03:00:00                      NaN                       NaN   
2016-01-01 04:00:00                      NaN                       NaN   

                     Brechhäuslau IN  Brechhäuslau OUT  Deffernik IN  \
Time                                                                   
2016-01-01 00:00:00              NaN               NaN           NaN   
2016-01-01 01:00:00              NaN               NaN           NaN   
2016-01-01 02:00:00              NaN               NaN           NaN   
2016-01-01 03:00:00              NaN               NaN           NaN   
2016-01-01 04:00:00              NaN             

In [289]:
# Slice the data from January 1, 2023, to August 19, 2024
df = df.loc['2023-01-01':'2024-08-19']
# Display the info to check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14328 entries, 2023-01-01 00:00:00 to 2024-08-19 23:00:00
Data columns (total 99 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Bayerisch Eisenstein IN                        14050 non-null  float64
 1   Bayerisch Eisenstein OUT                       14050 non-null  float64
 2   Brechhäuslau IN                                14049 non-null  float64
 3   Brechhäuslau OUT                               14049 non-null  float64
 4   Deffernik IN                                   13773 non-null  float64
 5   Deffernik OUT                                  13773 non-null  float64
 6   Diensthüttenstraße IN                          13906 non-null  float64
 7   Diensthüttenstraße OUT                         13906 non-null  float64
 8   Felswandergebiet IN                            13643 non-null  float64
 9   Felswandergebie

In [290]:
df.tail()

,Bayerisch Eisenstein IN,Bayerisch Eisenstein OUT,Brechhäuslau IN,Brechhäuslau OUT,Deffernik IN,Deffernik OUT,Diensthüttenstraße IN,Diensthüttenstraße OUT,Felswandergebiet IN,Felswandergebiet OUT,...,Scheuereck-Schachten-Trinkwassertalsperre IN,Scheuereck-Schachten-Trinkwassertalsperre OUT,ZScore_Daily_Max_Temperature (°C),ZScore_Daily_Max_Relative Humidity (%),ZScore_Daily_Max_Precipitation (mm),ZScore_Daily_Max_Wind Speed (km/h),ZScore_Daily_Max_Sunshine Duration (min),Distance_to_Nearest_Holiday_Bayern,Distance_to_Nearest_Holiday_CZ,Hour
Time,,,,,,,,,,,,,,,,,,,,,
2024-08-19 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.489326,0.535474,-0.065137,0.794311,-2.99305,4.0,40.0,19
2024-08-19 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.489326,0.535474,-0.065137,0.794311,-2.99305,4.0,40.0,20
2024-08-19 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.489326,0.535474,-0.065137,0.794311,-2.99305,4.0,40.0,21
2024-08-19 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.489326,0.535474,-0.065137,0.794311,-2.99305,4.0,40.0,22
2024-08-19 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.489326,0.535474,-0.065137,0.794311,-2.99305,4.0,40.0,23


In [291]:
region_columns = [
    'Falkenstein-Schwellhäusl IN', 
    'Rachel-Spiegelau IN', 
    'Nationalparkzentrum Falkenstein IN',
    'Nationalparkzentrum Lusen IN', 
    'Lusen-Mauth-Finsterau IN', 
    'Scheuereck-Schachten-Trinkwassertalsperre IN',
    'Falkenstein-Schwellhäusl OUT', 
    'Rachel-Spiegelau OUT', 
    'Nationalparkzentrum Falkenstein OUT',
    'Nationalparkzentrum Lusen OUT', 
    'Lusen-Mauth-Finsterau OUT', 
    'Scheuereck-Schachten-Trinkwassertalsperre OUT'
]

for column in region_columns:
    if column in df.columns:
        missing_dates = df[df[column].isna()].index
        if not missing_dates.empty:
            first_missing_date = missing_dates[0]
            print(f"Column '{column}' has its first missing value on {first_missing_date}")
        else:
            print(f"Column '{column}' has no missing values")
    else:
        print(f"Column '{column}' is not in the DataFrame")


Column 'Falkenstein-Schwellhäusl IN' has its first missing value on 2024-08-08 11:00:00
Column 'Rachel-Spiegelau IN' has its first missing value on 2024-08-07 14:00:00
Column 'Nationalparkzentrum Falkenstein IN' has no missing values
Column 'Nationalparkzentrum Lusen IN' has no missing values
Column 'Lusen-Mauth-Finsterau IN' has its first missing value on 2024-07-23 00:00:00
Column 'Scheuereck-Schachten-Trinkwassertalsperre IN' has its first missing value on 2024-08-06 18:00:00
Column 'Falkenstein-Schwellhäusl OUT' has its first missing value on 2024-08-08 11:00:00
Column 'Rachel-Spiegelau OUT' has its first missing value on 2024-08-07 14:00:00
Column 'Nationalparkzentrum Falkenstein OUT' has no missing values
Column 'Nationalparkzentrum Lusen OUT' has no missing values
Column 'Lusen-Mauth-Finsterau OUT' has its first missing value on 2024-07-23 00:00:00
Column 'Scheuereck-Schachten-Trinkwassertalsperre OUT' has its first missing value on 2024-08-06 18:00:00


In [292]:
# Slice the data from January 1, 2023, to July 22, 2024
df = df.loc['2023-01-01':'2024-07-22']
# Display the info to check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13656 entries, 2023-01-01 00:00:00 to 2024-07-22 23:00:00
Data columns (total 99 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Bayerisch Eisenstein IN                        13656 non-null  float64
 1   Bayerisch Eisenstein OUT                       13656 non-null  float64
 2   Brechhäuslau IN                                13656 non-null  float64
 3   Brechhäuslau OUT                               13656 non-null  float64
 4   Deffernik IN                                   13379 non-null  float64
 5   Deffernik OUT                                  13379 non-null  float64
 6   Diensthüttenstraße IN                          13656 non-null  float64
 7   Diensthüttenstraße OUT                         13656 non-null  float64
 8   Felswandergebiet IN                            13643 non-null  float64
 9   Felswandergebie

In [293]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

Deffernik IN                277
Deffernik OUT               277
Felswandergebiet IN          13
Felswandergebiet OUT         13
Fredenbrücke IN              13
Fredenbrücke OUT             13
Klingenbrunner Wald IN     1140
Klingenbrunner Wald OUT    1140
Sagwassersäge IN             13
Sagwassersäge OUT            13
Scheuereck IN               182
Scheuereck OUT              182
Schwarzbachbrücke IN         13
Schwarzbachbrücke OUT        13
Wistlberg IN                 35
Wistlberg OUT                35
Bucina IN                    12
Bucina OUT                   12
Lusen 1 IN                    3
Lusen 1 OUT                   3
dtype: int64


In [294]:
print(df.columns.tolist())

['Bayerisch Eisenstein IN', 'Bayerisch Eisenstein OUT', 'Brechhäuslau IN', 'Brechhäuslau OUT', 'Deffernik IN', 'Deffernik OUT', 'Diensthüttenstraße IN', 'Diensthüttenstraße OUT', 'Felswandergebiet IN', 'Felswandergebiet OUT', 'Ferdinandsthal IN', 'Ferdinandsthal OUT', 'Fredenbrücke IN', 'Fredenbrücke OUT', 'Gfäll IN', 'Gfäll OUT', 'Gsenget IN', 'Gsenget OUT', 'Klingenbrunner Wald IN', 'Klingenbrunner Wald OUT', 'Klosterfilz IN', 'Klosterfilz OUT', 'Racheldiensthütte IN', 'Racheldiensthütte OUT', 'Sagwassersäge IN', 'Sagwassersäge OUT', 'Scheuereck IN', 'Scheuereck OUT', 'Schillerstraße IN', 'Schillerstraße OUT', 'Schwarzbachbrücke IN', 'Schwarzbachbrücke OUT', 'Falkenstein 2 OUT', 'Falkenstein 2 IN', 'Lusen 2 IN', 'Lusen 2 OUT', 'Lusen 3 IN', 'Lusen 3 OUT', 'Waldhausreibe IN', 'Waldhausreibe OUT', 'Waldspielgelände IN', 'Waldspielgelände OUT', 'Wistlberg IN', 'Wistlberg OUT', 'Bucina IN', 'Bucina OUT', 'Falkenstein 1 IN', 'Falkenstein 1 OUT', 'Lusen 1 IN', 'Lusen 1 OUT', 'Trinkwasserta

In [295]:
# Create funtion for obtaining feature importances
def get_feature_names_after_encoding(preprocessor, numeric_features, categorical_features):
    # Get the transformer for the numeric features
    numeric_transformer = preprocessor.named_transformers_['num']
    
    # Get the transformer for the categorical features (OneHotEncoder)
    cat_transformer = preprocessor.named_transformers_['cat']
    
    # Get the feature names for numeric features (after scaling)
    numeric_feature_names = numeric_features
    
    # Get the feature names for categorical features (after one-hot encoding)
    cat_feature_names = cat_transformer.get_feature_names_out(categorical_features)
    
    # Combine all feature names
    feature_names = list(numeric_feature_names) + list(cat_feature_names)
    
    return feature_names


In [296]:
# Assuming df is your DataFrame with the data
# Define target and feature columns
target_vars = ['traffic_abs', 'sum_IN_abs', 'sum_OUT_abs', 'Lusen-Mauth-Finsterau IN', 'Lusen-Mauth-Finsterau OUT', 
               'Falkenstein-Schwellhäusl IN', 'Falkenstein-Schwellhäusl OUT', 'Rachel-Spiegelau IN', 
               'Rachel-Spiegelau OUT', 'Nationalparkzentrum Lusen IN', 'Nationalparkzentrum Lusen OUT', 
               'Scheuereck-Schachten-Trinkwassertalsperre IN', 'Scheuereck-Schachten-Trinkwassertalsperre OUT', 
               'Nationalparkzentrum Falkenstein IN', 'Nationalparkzentrum Falkenstein OUT']

numeric_features = ['Temperature (°C)', 'Relative Humidity (%)', 'Precipitation (mm)', 'Wind Speed (km/h)', 
                    'Sunshine Duration (min)', 'ZScore_Daily_Max_Temperature (°C)', 
                    'ZScore_Daily_Max_Relative Humidity (%)','ZScore_Daily_Max_Precipitation (mm)',
                    'ZScore_Daily_Max_Wind Speed (km/h)','ZScore_Daily_Max_Sunshine Duration (min)',
                    'Distance_to_Nearest_Holiday_Bayern','Distance_to_Nearest_Holiday_CZ']
categorical_features = ['Wochentag', 'Wochenende', 'Jahreszeit', 'Laubfärbung', 'Schulferien_Bayern', 
                        'Schulferien_CZ', 'Feiertag_Bayern', 'Feiertag_CZ', 'HEH_geoeffnet', 'HZW_geoeffnet', 'WGM_geoeffnet', 
                        'Lusenschutzhaus_geoeffnet', 'Racheldiensthuette_geoeffnet', 'Falkensteinschutzhaus_geoeffnet', 
                        'Schwellhaeusl_geoeffnet', ]

cyclic_features = ['Hour', 'Monat', 'Tag']

# Split the data into features and targets
X = df[numeric_features + categorical_features]
y = df[target_vars]



In [297]:
X.index

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2024-07-22 14:00:00', '2024-07-22 15:00:00',
               '2024-07-22 16:00:00', '2024-07-22 17:00:00',
               '2024-07-22 18:00:00', '2024-07-22 19:00:00',
               '2024-07-22 20:00:00', '2024-07-22 21:00:00',
               '2024-07-22 22:00:00', '2024-07-22 23:00:00'],
              dtype='datetime64[ns]', name='Time', length=13656, freq=None)

# Model Traning

In [329]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

class Preprocessor:
    def __init__(self, numeric_features, categorical_features, cyclic_features):
        self.numeric_features = numeric_features
        self.categorical_features = categorical_features
        self.cyclic_features = cyclic_features
        self.scaler = StandardScaler()
        self.encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
        self.fitted = False

    def fit(self, df):
        # Fit scaler on numeric features
        self.scaler.fit(df[self.numeric_features])

        # Generate cyclic features
        for feature in self.cyclic_features:
            df[f'{feature}_sin'] = np.sin(2 * np.pi * df[feature] / df[feature].max())
            df[f'{feature}_cos'] = np.cos(2 * np.pi * df[feature] / df[feature].max())

        # Fit encoder on categorical features (including cyclic ones)
        combined_features = self.categorical_features + [f'{f}_sin' for f in self.cyclic_features] + [f'{f}_cos' for f in self.cyclic_features]
        self.encoder.fit(df[combined_features])
        self.fitted = True

    def transform(self, df):
        if not self.fitted:
            raise RuntimeError("Preprocessor not fitted yet.")
        
        df_copy = df.copy()

        # Generate cyclic features
        for feature in self.cyclic_features:
            df_copy[f'{feature}_sin'] = np.sin(2 * np.pi * df_copy[feature] / df_copy[feature].max())
            df_copy[f'{feature}_cos'] = np.cos(2 * np.pi * df_copy[feature] / df_copy[feature].max())

        # Separate numeric and categorical features
        X_numeric = df_copy[self.numeric_features]
        X_categorical = df_copy[self.categorical_features + [f'{f}_sin' for f in self.cyclic_features] + [f'{f}_cos' for f in self.cyclic_features]]

        # Normalize numeric features
        X_numeric_scaled = self.scaler.transform(X_numeric)

        # Encode categorical features
        X_categorical_encoded = self.encoder.transform(X_categorical)

        # Combine features
        X = np.hstack([X_numeric_scaled, X_categorical_encoded])

        return X


In [310]:
# Define features
numeric_features = ['Temperature (°C)', 'Relative Humidity (%)', 'Precipitation (mm)', 'Wind Speed (km/h)', 
                    'Sunshine Duration (min)', 'ZScore_Daily_Max_Temperature (°C)', 
                    'ZScore_Daily_Max_Relative Humidity (%)', 'ZScore_Daily_Max_Precipitation (mm)',
                    'ZScore_Daily_Max_Wind Speed (km/h)', 'ZScore_Daily_Max_Sunshine Duration (min)',
                    'Distance_to_Nearest_Holiday_Bayern', 'Distance_to_Nearest_Holiday_CZ']
categorical_features = ['Wochentag', 'Wochenende', 'Jahreszeit', 'Laubfärbung', 'Schulferien_Bayern', 
                        'Schulferien_CZ', 'Feiertag_Bayern', 'Feiertag_CZ', 'HEH_geoeffnet', 'HZW_geoeffnet', 
                        'WGM_geoeffnet', 'Lusenschutzhaus_geoeffnet', 'Racheldiensthuette_geoeffnet', 
                        'Falkensteinschutzhaus_geoeffnet', 'Schwellhaeusl_geoeffnet']
cyclic_features = ['Tag', 'Monat', 'Hour']


In [330]:
# Initialize preprocessor
preprocessor = Preprocessor(numeric_features, categorical_features, cyclic_features)

In [331]:
# Fit preprocessor on training data
preprocessor.fit(df_train)

C:\Users\workshop\AppData\Local\Temp\ipykernel_9260\3079549517.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{feature}_sin'] = np.sin(2 * np.pi * df[feature] / df[feature].max())
C:\Users\workshop\AppData\Local\Temp\ipykernel_9260\3079549517.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{feature}_cos'] = np.cos(2 * np.pi * df[feature] / df[feature].max())
C:\Users\workshop\AppData\Local\Temp\ipykernel_9260\3079549517.py:19: SettingWithCopyWarning: 
A value is trying to be set on a co

In [311]:
# Example date ranges
train_end_date = "2023-12-31"
eval_start_date = "2024-05-01"
eval_end_date = "2024-07-22"

# Load your dataframe (assuming it's already loaded as 'df')
df_train = df.loc[df.index <= train_end_date]
df_eval = df.loc[(df.index >= eval_start_date) & (df.index <= eval_end_date)]


In [316]:
print(f"train shape: {df_train.shape}")
print(f"eval shape: {df_eval.shape}")

train shape: (8737, 99)
eval shape: (1969, 99)


In [332]:
# Transform training and evaluation data
X_train = preprocessor.transform(df_train)
X_eval = preprocessor.transform(df_eval)

c:\Users\workshop\.pyenv-win-venv\envs\bf_dssgx\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [16, 19] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [333]:
print(f"X train shape: {X_train.shape}")
print(f"X eval shape: {X_eval.shape}")

X train shape: (8737, 152)
X eval shape: (1969, 152)


In [334]:
# Define target columns (adjust as per your actual columns)
target_vars = ['traffic_abs', 'sum_IN_abs', 'sum_OUT_abs', 'Lusen-Mauth-Finsterau IN', 'Lusen-Mauth-Finsterau OUT', 
               'Falkenstein-Schwellhäusl IN', 'Falkenstein-Schwellhäusl OUT', 'Rachel-Spiegelau IN', 'Rachel-Spiegelau OUT', 
               'Nationalparkzentrum Lusen IN', 'Nationalparkzentrum Lusen OUT', 'Scheuereck-Schachten-Trinkwassertalsperre IN', 
               'Scheuereck-Schachten-Trinkwassertalsperre OUT', 'Nationalparkzentrum Falkenstein IN', 'Nationalparkzentrum Falkenstein OUT']

# Extract targets
y_train = df_train[target_vars].values
y_eval = df_eval[target_vars].values

In [335]:
print(f"y train shape: {y_train.shape}")
print(f"y eval shape: {y_eval.shape}")

y train shape: (8737, 15)
y eval shape: (1969, 15)


In [336]:
def create_sequences(X, y, window_size):
    X_seq, y_seq = [], []
    for i in range(len(X) - window_size):
        X_seq.append(X[i:i + window_size])
        y_seq.append(y[i + window_size])
    return np.array(X_seq), np.array(y_seq)

window_size = 168  # Example: 168 hours (7 days)
X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
X_eval_seq, y_eval_seq = create_sequences(X_eval, y_eval, window_size)

print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"X_eval_seq shape: {X_eval_seq.shape}")
print(f"y_eval_seq shape: {y_eval_seq.shape}")


X_train_seq shape: (8569, 168, 152)
y_train_seq shape: (8569, 15)
X_eval_seq shape: (1801, 168, 152)
y_eval_seq shape: (1801, 15)


In [338]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Model definition
model = Sequential()
model.add(Input(shape=input_shape))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(y_train_seq.shape[1]))

model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_seq, y_train_seq, epochs=50, batch_size=64, validation_split=0.1, callbacks=[early_stopping])


268/268 ━━━━━━━━━━━━━━━━━━━━ 53s 184ms/step - loss: 8325.5479 - val_loss: 7803.3271


In [346]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_performance(y_true_df, y_pred_df):
    """
    Evaluate the performance of the model using MAE, MSE, RMSE, R2, RMSLE, and MAPE.
    
    Parameters:
    y_true_df (pd.DataFrame): DataFrame with actual values.
    y_pred_df (pd.DataFrame): DataFrame with predicted values.
    
    Returns:
    pd.DataFrame: DataFrame containing the evaluation metrics for each target variable.
    """
    metrics = {}
    
    for col in y_true_df.columns:
        y_true = y_true_df[col]
        y_pred = y_pred_df[col]
        
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        rmsle = np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        
        metrics[col] = {
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2,
            'RMSLE': rmsle,
            'MAPE (%)': mape
        }
    
    metrics_df = pd.DataFrame(metrics).T
    return metrics_df


In [347]:
import plotly.graph_objects as go

def plot_actual_vs_predicted(y_test_df, y_pred_df, variables, date_index):
    """
    Plot actual vs predicted values for specified target variables using Plotly.
    
    Args:
    y_test_df (pd.DataFrame): The DataFrame with actual target values.
    y_pred_df (pd.DataFrame): The DataFrame with predicted target values.
    variables (list): List of target variables to plot.
    date_index (pd.DatetimeIndex): The index containing date and time.
    """
    for var in variables:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=date_index, y=y_test_df[var], mode='lines', name='Actual'))
        fig.add_trace(go.Scatter(x=date_index, y=y_pred_df[var], mode='lines', name='Predicted'))
        fig.update_layout(title=var, xaxis_title='Date', yaxis_title='Value')
        fig.show()


In [348]:
# Assume model.predict returns y_pred with the same shape as y_eval_seq
y_pred = model.predict(X_eval_seq)  # Shape should be (number of samples, number of target variables)

57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step


In [349]:
# Convert predictions to DataFrame with the same index as y_eval
y_pred_df = pd.DataFrame(y_pred, columns=target_vars)

In [350]:
# Ensure y_eval is also in DataFrame format with correct index
y_eval_df = pd.DataFrame(y_eval_seq, columns=target_vars)

In [351]:
# Evaluate performance
performance_df = evaluate_performance(y_eval_df, y_pred_df)
print(performance_df)

                                                      MAE           MSE  \
traffic_abs                                    156.921902  73657.804165   
sum_IN_abs                                      81.478722  18115.760649   
sum_OUT_abs                                     77.183855  16811.730673   
Lusen-Mauth-Finsterau IN                        15.401049   1170.882023   
Lusen-Mauth-Finsterau OUT                       17.123050    725.803174   
Falkenstein-Schwellhäusl IN                     17.598701    718.956319   
Falkenstein-Schwellhäusl OUT                    16.545509    608.547436   
Rachel-Spiegelau IN                             19.364275    842.308477   
Rachel-Spiegelau OUT                            16.139926    580.028001   
Nationalparkzentrum Lusen IN                    26.261063   1623.058227   
Nationalparkzentrum Lusen OUT                   24.430072   1514.293497   
Scheuereck-Schachten-Trinkwassertalsperre IN     4.926337     56.577477   
Scheuereck-Schachten-Trin

In [352]:
# Create DataFrames for actual and predicted values with the correct date index
date_index = df.index[-len(y_eval_df):]  # Adjust based on your evaluation index

In [353]:
# Plot actual vs predicted values
plot_actual_vs_predicted(y_eval_df, y_pred_df, target_vars, date_index)